# Teste de Ciência de Dados - Fase de mineração - Cinnecta

## Importação de bibliotecas

In [33]:
# Imports
import pandas as pd
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
import statsmodels.api as sm
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

random_state = 15

sns.set_theme(
    style="whitegrid",
    palette="dark",
    font="Segoe UI",
    font_scale=.75,
    context="notebook",
    rc={
        "figure.figsize": (12, 8),
        "figure.dpi": 80,
        },
)

## Importação de dados

In [34]:
data = pd.read_csv('airbnb_filtrado.csv')

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6387 entries, 0 to 6386
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   host_is_superhost            6387 non-null   bool   
 1   cancellation_policy          6387 non-null   object 
 2   instant_bookable             6387 non-null   bool   
 3   host_total_listings_count    6387 non-null   float64
 4   neighbourhood                6387 non-null   object 
 5   property_type                6387 non-null   object 
 6   room_type                    6387 non-null   object 
 7   accommodates                 6387 non-null   float64
 8   bathrooms                    6387 non-null   float64
 9   bedrooms                     6387 non-null   float64
 10  beds                         6387 non-null   float64
 11  bed_type                     6387 non-null   object 
 12  minimum_nights               6387 non-null   float64
 13  review_scores_rati

## Proposição

Considerando a análise dos dados, mediante análise direcionada, penso que é possível a criação de um modelo a partir dos dados que consiga predizer a quantidade de reservas que um imóvel pode ter dado um determinado conjunto de variáveis. Essa predição deve servir como uma estimativa para um conjunto de parâmetros usado pelo time de negócios de forma que possam focar seus esforços em um conjunto de imóveis possivelmente 'fracos'. Com essa visão de potenciais ganhos ou taxa de ocupação de imóveis já disponíveis na plataforma, outra alternativa seria a possibilidade de recomendar imóveis que provavelmente seriam mais reservados a usuários que desejam reservar um imóvel naquela localidade. Ou seja, a indicação de modelos provavelmente mais reserváveis a usuários buscando imóveis em determinada localidade.

A identificação dos atributos de um imóvel que faz um imóvel ser fraco pode ser feita através de análises mais detalhadas. A tabela de correlação é uma indicação dessa tendência, mas ela não informa se um atributo explica ou não a variável resposta `host_total_listings_count`. Essa análise é relativamente rápida de ser feita, mas não cobre a possibilidade de que atributos podem ter algum sentido em conjunto e separados podem ser ruins. Para isso essa análise deve ser repetida por várias combinações de atributos. 

Com essas considerações, escolho o RandomForest para o modelo de predição. RandomForest, como um algoritmo, treina diversas árvores de decisão com diferentes subconjuntos de instâncias e subconjuntos de atributos. Dessa forma um comitê com diferentes visões dos dados é criado. Dada a dificuldade da tarefa de regressão e a necessidade de testar diferentes subconjuntos de atributos, o RandomForest é uma boa escolha. Além disso, esse algoritmo pode informar a relevância dos atributos para a predição. Será a primeira análise que irei realizar.

## Execução do modelo

### Tabela de correlação

In [36]:
corr = data.corr(method='spearman')
corr.style.background_gradient(cmap='coolwarm').format("{:.2f}")

,host_is_superhost,instant_bookable,host_total_listings_count,accommodates,bathrooms,bedrooms,beds,minimum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,price
host_is_superhost,1.00,-0.10,-0.08,-0.01,-0.02,0.00,-0.02,-0.23,0.16,0.20,0.19,0.13,0.16,0.04,0.09,0.08
instant_bookable,-0.10,1.00,0.12,-0.06,-0.03,-0.07,-0.07,-0.08,-0.07,-0.04,-0.02,0.01,-0.03,0.01,0.00,-0.10
host_total_listings_count,-0.08,0.12,1.00,-0.11,0.01,-0.22,-0.10,0.17,-0.07,-0.11,-0.08,-0.09,-0.11,0.03,-0.00,-0.13
accommodates,-0.01,-0.06,-0.11,1.00,0.39,0.63,0.80,-0.05,0.04,0.03,0.02,0.02,0.02,-0.00,-0.05,0.58
bathrooms,-0.02,-0.03,0.01,0.39,1.00,0.48,0.41,0.04,0.05,-0.01,-0.02,-0.03,-0.01,-0.02,0.01,0.29
bedrooms,0.00,-0.07,-0.22,0.63,0.48,1.00,0.63,0.04,0.08,0.04,0.02,0.02,0.02,-0.03,-0.02,0.49
beds,-0.02,-0.07,-0.10,0.80,0.41,0.63,1.00,-0.03,0.01,-0.00,-0.02,-0.01,0.00,-0.03,-0.06,0.47
minimum_nights,-0.23,-0.08,0.17,-0.05,0.04,0.04,-0.03,1.00,0.05,-0.04,0.00,-0.07,-0.04,0.04,0.06,-0.01
review_scores_rating,0.16,-0.07,-0.07,0.04,0.05,0.08,0.01,0.05,1.00,0.45,0.51,0.30,0.37,0.32,0.52,0.22
review_scores_accuracy,0.20,-0.04,-0.11,0.03,-0.01,0.04,-0.00,-0.04,0.45,1.00,0.53,0.42,0.50,0.30,0.46,0.16


In [37]:
data.columns

Index(['host_is_superhost', 'cancellation_policy', 'instant_bookable',
       'host_total_listings_count', 'neighbourhood', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'bed_type', 'minimum_nights', 'review_scores_rating',
       'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'price'],
      dtype='object')

### Criação da pipeline de tratamento das colunas

In [38]:
# Definição das colunas que serão utilizadas no modelo
categorical = [
    'host_is_superhost', 
    'cancellation_policy', 
    'instant_bookable',
    'neighbourhood', 
    'property_type',
    'room_type', 
    'bed_type', 
    ]

numerical = [
    'accommodates', 
    'bathrooms', 
    'bedrooms', 
    'beds',
    'minimum_nights', 
    'review_scores_rating',
    'review_scores_accuracy', 
    'review_scores_cleanliness',
    'review_scores_checkin', 
    'review_scores_communication',
    'review_scores_location', 
    'review_scores_value', 
    'price'
    ]

# Definição da variável resposta
target = 'host_total_listings_count'

In [39]:

def create_regression_pipeline(categorical: list, numerical: list) -> Pipeline:
    """create_regression_pipeline: Criação do pipeline de regressão.

    Args:
        categorical (list): lista de variáveis categóricas
        numerical (list): lista de variáveis numéricas

    Returns:
        Pipeline: pipeline de regressão com transformação de colunas e o modelo de regressão.
    """
    # Criação do transformer de colunas
    column_transformer = ColumnTransformer(
        [
            ('categorical', OneHotEncoder(drop='first', categories='auto', handle_unknown='ignore'), categorical),
            ('numerical', StandardScaler(), numerical)
        ],
        remainder='drop',
        )

    reg = RandomForestRegressor(
        n_estimators=100, random_state=random_state, n_jobs=8)
    model = Pipeline([
        ('column_transformer', column_transformer),
        ('regressor', reg)
    ])
    return model

model = create_regression_pipeline(categorical, numerical)

### Separação dos dados em treino e teste

In [40]:
X = data.drop(columns=target)
y = data[target]

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state, shuffle=True)

### Análise de significância

In [42]:
X_temp = model[0].fit_transform(X_train)
X_temp = pd.DataFrame(X_temp.toarray(), columns=model[0].get_feature_names_out())

In [43]:
X_temp = sm.add_constant(X_temp)
ols = sm.OLS(y_train.values, X_temp)
results = ols.fit()
results.pvalues[results.pvalues < 0.05]

categorical__host_is_superhost_True                  5.286517e-13
categorical__cancellation_policy_moderate            1.143575e-07
categorical__cancellation_policy_strict             6.925819e-110
categorical__neighbourhood_Downtown/Civic Center     6.357475e-05
categorical__neighbourhood_Financial District        7.106338e-07
categorical__neighbourhood_Nob Hill                  6.727273e-03
categorical__neighbourhood_Potrero Hill              3.398732e-02
categorical__neighbourhood_South of Market           1.905088e-19
categorical__neighbourhood_Western Addition          8.557775e-04
categorical__property_type_Apartment                 3.760485e-02
categorical__property_type_Bed and breakfast         3.480363e-03
categorical__property_type_Hotel                     7.382830e-03
categorical__room_type_Private room                  3.219843e-28
categorical__room_type_Shared room                   2.213727e-06
numerical__beds                                      5.776439e-04
numerical_

Tais variáveis conseguem explicar o modelo. As outras variáveis não são significativas, mas manterei elas no modelo por enquanto.

### Primeiro `fit` e feature importance

In [44]:
model.fit(X_train, y_train)

Pipeline(steps=[('column_transformer',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['host_is_superhost',
                                                   'cancellation_policy',
                                                   'instant_bookable',
                                                   'neighbourhood',
                                                   'property_type', 'room_type',
                                                   'bed_type']),
                                                 ('numerical', StandardScaler(),
                                                  ['accommodates', 'bathrooms',
                                                   'bedrooms', 'beds',
                                                   'minimum_nights',
                                                   'review_scores_rating',
                                                   'review_scores_accuracy',
                                                   'review_scores_cleanliness',
                                                   'review_scores_checkin',
                                                   'review_scores_communication',
                                                   'review_scores_location',
                                                   'review_scores_value',
                                                   'price'])])),
                ('regressor',
                 RandomForestRegressor(n_jobs=8, random_state=15))])

In [45]:
model.score(X_test, y_test)

0.6845233392498462

Apesar da baixa correlação com as variáveis e poucas variáveis significativas o modelo performou bem no conjunto de teste. Agora, a visualização da feature importance.

#### Feature Importance

In [46]:
importances = model[1].feature_importances_
feature_names = model[0].get_feature_names_out()
importances = pd.DataFrame({'feature': feature_names, 'importance': importances})
px.histogram(importances, x='importance', text_auto=True)

Vejamos as importances das variáveis fora do primeiro grupo:

In [47]:
pd.set_option('display.max_rows', None)
importances.loc[importances.importance >= 0.01].sort_values(by='importance', ascending=False)

,feature,importance
81,numerical__price,0.200505
2,categorical__cancellation_policy_strict,0.095357
73,numerical__minimum_nights,0.084192
0,categorical__host_is_superhost_True,0.073175
74,numerical__review_scores_rating,0.062060
6,categorical__instant_bookable_True,0.060347
37,categorical__neighbourhood_South of Market,0.060284
1,categorical__cancellation_policy_moderate,0.057696
42,categorical__property_type_Apartment,0.056546
69,numerical__accommodates,0.032657


Das variáveis escolhidas, algumas não apareceram fora desse grupo, mas, como esperado, bed_type foi um atributo que não apareceu. Observando o restante das variáveis:

In [48]:
importances.loc[importances.importance < 0.01].sort_values(
    by='importance', ascending=False)


,feature,importance
12,categorical__neighbourhood_Downtown/Civic Center,7.177203e-03
70,numerical__bathrooms,6.141714e-03
78,numerical__review_scores_communication,5.076942e-03
7,categorical__neighbourhood_Bernal Heights,4.886997e-03
76,numerical__review_scores_cleanliness,4.424835e-03
24,categorical__neighbourhood_Noe Valley,3.886025e-03
32,categorical__neighbourhood_Potrero Hill,3.870973e-03
25,categorical__neighbourhood_North Beach,3.796474e-03
22,categorical__neighbourhood_Mission,3.683813e-03
47,categorical__property_type_Condominium,3.251011e-03


Nas posições mais baixas temos atributos categóricos com pouca importância. O tipo de cama aparece em posições variadas, mas ainda tem algum impacto. Um teste removendo alguns atributos menos importantes:

In [49]:
categorical.remove('bed_type')
numerical.remove('review_scores_accuracy')
numerical.remove('review_scores_checkin',)

In [50]:
model = create_regression_pipeline(categorical, numerical)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.6833167456929667

O teste não parece ter afetado tanto o desempenho e até mesmo diminuiu ligeiramente. Ainda assim, o modelo é bom. Vou redefinir o conjunto de variáveis antes de configurar o GridSearch.

In [51]:
categorical = [
    'host_is_superhost',
    'cancellation_policy',
    'instant_bookable',
    'neighbourhood',
    'property_type',
    'room_type',
    'bed_type',
]

numerical = [
    'accommodates',
    'bathrooms',
    'bedrooms',
    'beds',
    'minimum_nights',
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value',
    'price'
]


### GridSearchCV

In [52]:
gs = GridSearchCV(
    estimator=create_regression_pipeline(categorical, numerical),
    param_grid={
        'regressor__max_depth': [None, 8],
        'regressor__min_samples_leaf': [1, 7],
        'regressor__min_samples_split': [2, 5, 10],
        'regressor__max_features': [0.5, 1.0],
        'regressor__criterion': ['squared_error', 'poisson', 'absolute_error'],
    },
    cv=3,
    scoring='neg_mean_absolute_error',
    verbose=0,
    n_jobs=1,
)
gs.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('column_transformer',
                                        ColumnTransformer(transformers=[('categorical',
                                                                         OneHotEncoder(drop='first',
                                                                                       handle_unknown='ignore'),
                                                                         ['host_is_superhost',
                                                                          'cancellation_policy',
                                                                          'instant_bookable',
                                                                          'neighbourhood',
                                                                          'property_type',
                                                                          'room_type',
                                                                          'bed_type']),
                                                                        ('numerical',
                                                                         StandardScaler(),
                                                                         ['accommodates',
                                                                          'bathrooms',
                                                                          'bedroom...
                                                                          'review_scores_value',
                                                                          'price'])])),
                                       ('regressor',
                                        RandomForestRegressor(n_jobs=8,
                                                              random_state=15))]),
             n_jobs=1,
             param_grid={'regressor__criterion': ['squared_error', 'poisson',
                                                  'absolute_error'],
                         'regressor__max_depth': [None, 8],
                         'regressor__max_features': [0.5, 1.0],
                         'regressor__min_samples_leaf': [1, 7],
                         'regressor__min_samples_split': [2, 5, 10]},
             scoring='neg_mean_absolute_error')

In [53]:
gs.score(X_test, y_test)

-27.55509335461358

In [54]:
gs.best_estimator_.score(X_test, y_test)

0.6519647121480046

O melhor conjunto de parâmetros é:

In [55]:
gs.best_params_

{'regressor__criterion': 'poisson',
 'regressor__max_depth': None,
 'regressor__max_features': 1.0,
 'regressor__min_samples_leaf': 1,
 'regressor__min_samples_split': 2}

Mais testes e parâmetros talvez fossem necessários para melhorarar o desempenho do regressor.

### Considerações finais sobre o modelo de regressão

O modelo de regressão performa relativamente bem, com um r2 de `0.68`. O MAE está com um valor que pode ser aceitável para o conjunto de dados, visto que 70% dos dados estão em um intervalo de 0 e 18 reservas. O restante dos imóveis destoa desse intervalo. O método de filtragem pelo IQR é uma opção para tratar os pontos que podem ser considerados outliers. Fazer isso pode diminuir o erro do modelo e gerar mais confiança.

In [56]:
px.histogram(data, x='host_total_listings_count',
            title='Host Total Listings', marginal='box')


### Discretização dos dados

Uma estratégia que pode ser adotada pelo time de negócios é de gerar uma escala de classificação de imóveis em relação à taxa de ocupação/reservas. Para isso, os dados de reserva devem ser discretizados de acordo com a escala que se deseja criar.

Irei discretizar os dados com a criação de 3 grupos de intervalos iguais, para simplicidade. Um time de negócios com maior conhecimento de negócio pode definir uma escala de classificação de reservas de imóveis.

In [57]:
# Discretizando a variável hotel_total_listings_count
y = pd.cut(
    data.host_total_listings_count,
    bins=3,
    labels=['low', 'medium', 'high'],
)

Histograma com os dados discretizados

In [58]:
px.histogram(data, x='host_total_listings_count', title='Host Total Listings', color=y)

Aqui defino a pipeline para um modelo de classificação:

In [59]:
def create_classification_pipeline(categorical: list, numerical: list) -> Pipeline:
    """_summary_

    Args:
        categorical (list): lista de variáveis categóricas
        numerical (list): lista de variáveis numéricas

    Returns:
        Pipeline: Pipeline para um modelo de classificação composta de transformação das colunas e do modelo de classificação.
    """
    
    column_transformer = ColumnTransformer(
        [
            ('categorical', OneHotEncoder(drop='first', categories='auto', handle_unknown='ignore'), categorical),
            ('numerical', StandardScaler(), numerical)
        ],
        remainder='drop',
    )

    reg = RandomForestClassifier(
        n_estimators=100, random_state=random_state, n_jobs=8)
    model = Pipeline([
        ('column_transformer', column_transformer),
        ('regressor', reg)
    ])
    return model

model = create_classification_pipeline(categorical, numerical)

Irei sobrescrever as variáveis de entrada para o modelo de classificação visto que não irei reutilizá-las e o split será diferente.

In [60]:
X = data.drop(columns=target)

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state, stratify=y)

In [62]:
model.fit(X_train, y_train)

Pipeline(steps=[('column_transformer',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'),
                                                  ['host_is_superhost',
                                                   'cancellation_policy',
                                                   'instant_bookable',
                                                   'neighbourhood',
                                                   'property_type', 'room_type',
                                                   'bed_type']),
                                                 ('numerical', StandardScaler(),
                                                  ['accommodates', 'bathrooms',
                                                   'bedrooms', 'beds',
                                                   'minimum_nights',
                                                   'review_scores_rating',
                                                   'review_scores_accuracy',
                                                   'review_scores_cleanliness',
                                                   'review_scores_checkin',
                                                   'review_scores_communication',
                                                   'review_scores_location',
                                                   'review_scores_value',
                                                   'price'])])),
                ('regressor',
                 RandomForestClassifier(n_jobs=8, random_state=15))])

In [63]:
y_hat = model.predict(X_test)
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

        high       0.86      0.67      0.76        46
         low       0.98      1.00      0.99      1206
      medium       1.00      0.69      0.82        26

    accuracy                           0.98      1278
   macro avg       0.95      0.79      0.85      1278
weighted avg       0.98      0.98      0.98      1278



Como visto o desempenho da classificação, o modelo é bom -- tanto na micro quanto na macro F1. Vou definir um pequeno GridSearch para verificar se consigo melhorar esses resultados.

In [64]:
gs = GridSearchCV(
    estimator=create_classification_pipeline(categorical, numerical),
    param_grid={
        'regressor__n_estimators': [100, 300],
        'regressor__max_depth': [None, 8],
        'regressor__min_samples_leaf': [1, 7],
        'regressor__min_samples_split': [2, 5],
        'regressor__max_features': [0.5, 1.0],
    },
    cv=3,
    scoring=['f1_micro', 'f1_macro'],
    refit='f1_micro',
    verbose=0,
    n_jobs=1,
)
gs.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('column_transformer',
                                        ColumnTransformer(transformers=[('categorical',
                                                                         OneHotEncoder(drop='first',
                                                                                       handle_unknown='ignore'),
                                                                         ['host_is_superhost',
                                                                          'cancellation_policy',
                                                                          'instant_bookable',
                                                                          'neighbourhood',
                                                                          'property_type',
                                                                          'room_type',
                                                                          'bed_type']),
                                                                        ('numerical',
                                                                         StandardScaler(),
                                                                         ['accommodates',
                                                                          'bathrooms',
                                                                          'bedroom...
                                                                          'review_scores_location',
                                                                          'review_scores_value',
                                                                          'price'])])),
                                       ('regressor',
                                        RandomForestClassifier(n_jobs=8,
                                                               random_state=15))]),
             n_jobs=1,
             param_grid={'regressor__max_depth': [None, 8],
                         'regressor__max_features': [0.5, 1.0],
                         'regressor__min_samples_leaf': [1, 7],
                         'regressor__min_samples_split': [2, 5],
                         'regressor__n_estimators': [100, 300]},
             refit='f1_micro', scoring=['f1_micro', 'f1_macro'])

In [65]:
gs.score(X_test, y_test)

0.9835680751173709

In [66]:
gs.best_params_

{'regressor__max_depth': None,
 'regressor__max_features': 0.5,
 'regressor__min_samples_leaf': 1,
 'regressor__min_samples_split': 2,
 'regressor__n_estimators': 100}

In [67]:
print(classification_report(y_test, gs.predict(X_test)))

              precision    recall  f1-score   support

        high       0.87      0.74      0.80        46
         low       0.99      1.00      0.99      1206
      medium       1.00      0.85      0.92        26

    accuracy                           0.98      1278
   macro avg       0.95      0.86      0.90      1278
weighted avg       0.98      0.98      0.98      1278



Com o GridSearch para o modelo de classificação os resultados melhoraram para a métrica macro f1. Concluo que foram bons resultados.

### Considerações finais sobre o modelo de classificação

O modelo de classificação, aqui, talvez seja o mais indicado para o time de negócios a identificar imóveis com características de alta ocupação. O modelo alcançou bons resultados, acima de 90% de acurácia e teve dificuldades em classificar os imóveis com alta ocupação, mas no geral performa muito bem. Concluo que a discretização foi uma boa opção para tratar esse problema, mas possui suas ressalvas.

A desvantagem principal ao utilizar a técnica da discretização é a perda de informação. Transformar os valores em faixas de valores facilita o trabalho do modelo justamente por generalizar a informação que é passada como alvo para o modelo. Então, apenas teremos aqui a predição de qual faixa de valores melhor representa dado imóvel. No entanto, a resolução ou intervalos de geração das faixas de valores pode ser ajustada pelo time de negócios para melhor atender a demanda.

## Considerações finais sobre o teste e ideias para produtização do modelo

Para produtização do modelo tenho a seguinte ideia:

Os modelos gerados podem auxiliar o time de vendas ao predizer pontuações ou faixas de ocupação para os imóveis e possibilitar um foco maior em imóveis que possam não performar tão bem ou possibilitar um incentivo a imóveis que apresentam um potencial de crescimento acima da faixa de ocupação na qual se encontra. No entanto, esses modelos poderiam ser melhor aproveitados na implantação de um sistema de recomendação. Dado que um cliente qualquer demonstra interesse em alguma localidade, o sistema de recomendação com esse modelo e esses dados seria usado para mostrar ao cliente os imóveis que tem maior potencial de reserva e não só os mais reservados até o momento. Isso aumentaria o alcance de imóveis mais novos na plataforma e aumentaria a taxa de ocupação e rentabilidade ao mostrar opções atrativas aos clientes. Uma forma importante de melhorar esse sistema é tendo maior disponibilidade de dados para que o modelo consiga um melhor ajuste e desempenho na tarefa de recomendação, em específico. Os dados de perfil de cliente, por exemplo, poderiam ser muito úteis para direcionar imóveis que possam ser mais atrativos a determinados clientes com padrões de negociação diferentes.

Quanto à realização do teste entendo que dei o meu melhor. O prazo para a realização do teste foi curto e tive que limitar as analises mais longas. Com mais dados a tarefa seria mais fácil, acredito, mas acredito ter cumprido o propósito e objetivo do teste.